In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Convert categorical columns to numerical values
# Map 'sex' and 'smoker' to binary values (0 or 1)
dataset['sex'] = dataset['sex'].map({'female': 0, 'male': 1})
dataset['smoker'] = dataset['smoker'].map({'no': 0, 'yes': 1})
# One-hot encode 'region' to create binary columns for each region
dataset = pd.get_dummies(dataset, columns=['region'], prefix='region')

# Split dataset into 80% train and 20% test
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

# Pop 'expenses' column to create labels
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

# Normalize numerical features (age, bmi, children) for better model performance
# Create a normalization layer to scale features to mean=0, std=1
normalizer = layers.experimental.preprocessing.Normalization()
# Adapt normalizer to training data (exclude labels)
normalizer.adapt(np.array(train_dataset))

# Build the neural network model
model = keras.Sequential([
    # Input layer with normalization
    normalizer,
    # Hidden layers with ReLU activation to capture non-linear relationships
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    # Output layer: single unit for regression (predicting expenses)
    layers.Dense(1)
])

# Compile the model
model.compile(
    optimizer='adam',  # Adam optimizer for efficient gradient descent
    loss='mean_absolute_error',  # MAE as loss to optimize for challenge metric
    metrics=['mae', 'mse']  # Track MAE and MSE during training
)

# Train the model
history = model.fit(
    train_dataset,
    train_labels,
    epochs=100,  # Train for 100 epochs to converge
    validation_split=0.2,  # Use 20% of training data for validation
    verbose=0  # Suppress training output for cleaner notebook
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
